Importing modules

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import tensorflow as tf

from sklearn.metrics import r2_score

Retrieving Data

In [3]:
data = pd.read_csv('startup_funding.csv')
#Viewing data
# data

,SNo,Date,StartupName,IndustryVertical,SubVertical,CityLocation,InvestorsName,InvestmentType,AmountInUSD,Remarks
0,0,01-08-2017,TouchKin,Technology,Predictive Care Platform,Bangalore,Kae Capital,Private Equity,"13,00,000",NaN
1,1,02-08-2017,Ethinos,Technology,Digital Marketing Agency,Mumbai,Triton Investment Advisors,Private Equity,NaN,NaN
2,2,02-08-2017,Leverage Edu,Consumer Internet,Online platform for Higher Education Services,New Delhi,"Kashyap Deorah, Anand Sankeshwar, Deepak Jain,...",Seed Funding,NaN,NaN
3,3,02-08-2017,Zepo,Consumer Internet,DIY Ecommerce platform,Mumbai,"Kunal Shah, LetsVenture, Anupam Mittal, Hetal ...",Seed Funding,"5,00,000",NaN
4,4,02-08-2017,Click2Clinic,Consumer Internet,healthcare service aggregator,Hyderabad,"Narottam Thudi, Shireesh Palle",Seed Funding,"8,50,000",NaN
...,...,...,...,...,...,...,...,...,...,...
2367,2367,29-01-2015,Printvenue,NaN,NaN,NaN,Asia Pacific Internet Group,Private Equity,"45,00,000",NaN
2368,2368,29-01-2015,Graphene,NaN,NaN,NaN,KARSEMVEN Fund,Private Equity,"8,25,000",Govt backed VC Fund
2369,2369,30-01-2015,Mad Street Den,NaN,NaN,NaN,"Exfinity Fund, GrowX Ventures.",Private Equity,"15,00,000",NaN
2370,2370,30-01-2015,Simplotel,NaN,NaN,NaN,MakeMyTrip,Private Equity,NaN,"Strategic Funding, Minority stake"


Pre-processing

In [17]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop ID and high-cardinality columns
    df = df.drop(['SNo', 'StartupName', 'SubVertical', 'InvestorsName'], axis=1)
    
    # Clean \\xc2\\xa0 examples
    df = df.applymap(lambda x: x.replace(r'\\xc2\\xa0', '') if type(x) == str else x)
    
    # Clean target column
    df['AmountInUSD'] = df['AmountInUSD'].apply(lambda x: x.replace(',', '') if str(x) != 'nan' else x)
    df['AmountInUSD'] = df['AmountInUSD'].replace({
        'undisclosed': np.NaN,
        'unknown': np.NaN,
        'Undisclosed': np.NaN,
        'N/A': np.NaN,
        '14342000+': '14342000'
    })
    
    # Drop missing target rows
    missing_target_rows = df[df['AmountInUSD'].isna()].index
    df = df.drop(missing_target_rows, axis=0).reset_index(drop=True)
    
    # Drop columns with more than 25% missing values
    df = df.drop('Remarks', axis=1)
    
    # Fill categorical missing values with most frequent occurence
    for column in ['IndustryVertical', 'CityLocation', 'InvestmentType']:
        df[column] = df[column].fillna(df[column].mode()[0])
    
    # Clean date column
    df['Date'] = df['Date'].replace({
        '05/072018': '05/07/2018',
        '01/07/015': '01/07/2015',
        '22/01//2015': '22/01/2015'
    })
    
    # Extract date features
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].apply(lambda x: x.year)
    df['Month'] = df['Date'].apply(lambda x: x.month)
    df['Day'] = df['Date'].apply(lambda x: x.day)
    df = df.drop('Date', axis=1)
    
    # Convert target column to float
    df['AmountInUSD'] = df['AmountInUSD'].astype(float)
    
    # Split df into X and y
    y = df['AmountInUSD']
    X = df.drop('AmountInUSD', axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    return X_train, X_test, y_train, y_test

In [18]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [19]:
# Viewing train set X
# X_train

,IndustryVertical,CityLocation,InvestmentType,Year,Month,Day
1453,Consumer Internet,Bangalore,Private Equity,2015,5,2
275,Technology,Pune,Private Equity,2017,1,25
1130,mobile payment software platform,Bangalore,Private Equity,2015,10,8
1153,"Hyperlocal Maintenance, Repair & Cleaning serv...",Mumbai,Private Equity,2015,8,19
1172,CRM / Analytics platform,Chennai,Seed Funding,2015,1,7
...,...,...,...,...,...,...
715,Consumer Internet,Bangalore,Seed Funding,2016,7,3
905,Virtual Health consultation app,Mumbai,Private Equity,2015,12,16
1096,Employee OnBoarding & Orientation platform,Noida,Seed Funding,2015,9,24
235,Consumer Internet,Gurgaon,Seed Funding,2017,2,18


In [20]:
# Viewing train set Y
# y_train

1453    18000000.0
275      4000000.0
1130     1000000.0
1153     2400000.0
1172     1000000.0
           ...    
715       500000.0
905      1200000.0
1096      400000.0
235        40000.0
1061      770000.0
Name: AmountInUSD, Length: 1067, dtype: float64

Building Pipeline

In [34]:
def build_model():
    inputs = tf.keras.Input(shape=(422,))
    x = tf.keras.layers.Dense(128, activation='relu')(inputs)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='linear')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer='adam',
        loss='mse'
    )
    
    return model

In [ ]:
nominal_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('nominal', nominal_transformer, ['IndustryVertical', 'CityLocation', 'InvestmentType'])
], remainder='passthrough')

regressor = tf.keras.wrappers.scikit_learn.KerasRegressor(build_model)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler()),
    ('regressor', regressor)
])

Training

In [36]:
model.fit(
    X_train,
    y_train,
    regressor__validation_split=0.2,
    regressor__batch_size=32,
    regressor__epochs=100,
    regressor__callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=1,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
27/27 [==============================] - 3s 11ms/step - loss: 4570265572868096.0000 - val_loss: 669353942974464.0000
Epoch 2/100
27/27 [==============================] - 0s 3ms/step - loss: 4570265572868096.0000 - val_loss: 669353674539008.0000
Epoch 3/100
27/27 [==============================] - 0s 4ms/step - loss: 4570265572868096.0000 - val_loss: 669353204776960.0000
Epoch 4/100
27/27 [==============================] - 0s 4ms/step - loss: 4570263425384448.0000 - val_loss: 669352063926272.0000
Epoch 5/100
27/27 [==============================] - 0s 4ms/step - loss: 4570259667288064.0000 - val_loss: 669350117769216.0000
Epoch 6/100
27/27 [==============================] - 0s 3ms/step - loss: 4570254298578944.0000 - val_loss: 669347030761472.0000
Epoch 7/100
27/27 [==============================] - 0s 4ms/step - loss: 4570245708644352.0000 - val_loss: 669342266032128.0000
Epoch 8/100
27/27 [==============================] - 0s 3ms/step - loss: 4570233897484288.0000 - val_lo

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('nominal',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['IndustryVertical',
                                                   'CityLocation',
                                                   'InvestmentType'])])),
                ('scaler', StandardScaler()),
                ('regressor',
                 <keras.wrappers.scikit_learn.KerasRegressor object at 0x0000017D65AF22E0>)])

Results

In [37]:
y_pred = model.predict(X_test)

rmse = np.sqrt(np.mean((y_test - y_pred)**2))
print("     Test RMSE: {:.2f}".format(rmse))

r2 = r2_score(y_test, y_pred)
print("Test R^2 Score: {:.5f}".format(r2))

15/15 [==============================] - 0s 1ms/step
     Test RMSE: 72405476.35
Test R^2 Score: -0.01587
